# Serial Execution

Resnet 50 Model with 10 CPUs for ResNet50

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torchvision.models import resnet50, ResNet50_Weights
import time
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
num_cpu_cores = torch.get_num_threads()
print("Number of CPU cores:", num_cpu_cores)

Number of CPU cores: 10


In [3]:
data_dir = '/home/hindupur.v/varsha_hpp/dataset/lung_colon_image_set'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
num_ftrs = model.fc.in_features
num_classes = 5
model.fc = nn.Linear(num_ftrs, num_classes)  # 5 classes
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class LungColonCancerDataset(datasets.ImageFolder):
    def __init__(self, root_dir, transform=None):
        super(LungColonCancerDataset, self).__init__(root=root_dir, transform=transform)


In [7]:
def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            
            # Use AMP for evaluation
            with amp.autocast():
                outputs = model(inputs)
            
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1

In [8]:
def plot_learning_curves(train_losses, valid_losses, valid_accuracies):
    plt.plot(train_losses, label='Train Loss')
    plt.plot(valid_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()

    plt.plot(valid_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Validation Accuracy')
    plt.show()

In [9]:
def train(num_workers, batch_size):
    print("Starting training...")

    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # train dataset
    train_dataset = LungColonCancerDataset(root_dir=os.path.join(data_dir, 'train'), transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

    # valid dataset
    validation_dataset = LungColonCancerDataset(root_dir=os.path.join(data_dir, 'valid'), transform=transform)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)
    
    # test dataset
    test_dataset = LungColonCancerDataset(root_dir=os.path.join(data_dir, 'test'), transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    train_losses = []
    valid_losses = []
    valid_accuracies = []

    start_time = time.time()
    print("Model Training Start Time:", start_time)

    for epoch in range(10):  # number of epochs
        epoch_start_time = time.time()  # Start time for the epoch
        running_loss = 0.0
        epoch_train_losses = []
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_train_losses.append(loss.item())  # Appending the loss for all the batch

            if batch_idx % 10 == 0:  # Log every 10 batches
                print(f"Epoch {epoch+1} Batch {batch_idx+1}: Loss {loss.item()}")

        train_losses.append(sum(epoch_train_losses) / len(epoch_train_losses))

        epoch_duration = time.time() - epoch_start_time
        print("Each Epoch Duration:", epoch_duration)

        # Validation
        model.eval()
        valid_loss = 0.0
        correct = 0
        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += torch.sum(preds == labels.data).item()

        valid_loss = valid_loss / len(validation_loader.dataset)
        valid_acc = correct / len(validation_loader.dataset)
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_acc)
        print(f'Epoch {epoch+1}, Validation Loss: {valid_loss}, Validation Acc: {valid_acc}')

    print("Test Results:")
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data).item()

    test_loss = test_loss / len(test_loader.dataset)
    test_acc = correct / len(test_loader.dataset)
    print(f'Test Loss: {test_loss}, Test Acc: {test_acc}')

    end_time = time.time()
    print("Model Training End Time:", end_time)

    duration = end_time - start_time
    print("Training duration:", duration, "seconds")

    return model


In [10]:
# Calling the training function to get the trained model executing serially
num_workers = 10
batch_size = 1000
trained_model = train(num_workers, batch_size)

Starting training...
Model Training Start Time: 1713140904.6691897
Epoch 1 Batch 1: Loss 1.6283828020095825
Epoch 1 Batch 11: Loss 1.4991755485534668
Each Epoch Duration: 536.9487495422363
Epoch 1, Validation Loss: 0.0016608821868896485, Validation Acc: 0.648
Epoch 2 Batch 1: Loss 1.3676106929779053
Epoch 2 Batch 11: Loss 0.6670449376106262
Each Epoch Duration: 495.1871888637543
Epoch 2, Validation Loss: 0.0002649354964494705, Validation Acc: 0.9412
Epoch 3 Batch 1: Loss 0.19778181612491608
Epoch 3 Batch 11: Loss 0.09604158997535706
Each Epoch Duration: 503.80170035362244
Epoch 3, Validation Loss: 0.00010097379088401794, Validation Acc: 0.97
Epoch 4 Batch 1: Loss 0.06907369941473007
Epoch 4 Batch 11: Loss 0.06343428790569305
Each Epoch Duration: 491.56688475608826
Epoch 4, Validation Loss: 7.198519110679626e-05, Validation Acc: 0.9788
Epoch 5 Batch 1: Loss 0.047100260853767395
Epoch 5 Batch 11: Loss 0.03570093959569931
Each Epoch Duration: 497.52460265159607
Epoch 5, Validation Loss: 7

Resnet 50 Model with 10 CPUs for Resnet 50

Average Execution time for each Epoch:

Epoch 1: 536.9487495422363

Epoch 2: 495.1871888637543

Epoch 3: 503.8017003536224

Epoch 4: 491.5668847560882

Epoch 5: 497.5246026515961

Epoch 6: 502.8919975757599

Epoch 7: 492.20523381233215

Epoch 8: 492.1724388599396

Epoch 9: 491.6890709400177

Epoch 10: 491.2844989299774


The average combined execution time for all epochs is 497.09 seconds.